# Text Generation using T5

* This tutorial is intended to provide, a familiarity in how to use ```T5``` for text-generation tasks.
* No training is involved in this.

In [ ]:
!pip install tf-transformers

!pip install transformers


In [19]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Supper TF warnings

import tensorflow as tf
print("Tensorflow version", tf.__version__)
print("Devices", tf.config.list_physical_devices())

from tf_transformers.models import T5Model, T5TokenizerTFText
from tf_transformers.core import TextGenerationChainer
from tf_transformers.text import TextDecoder, TextDecoderSerializable

Tensorflow version 2.7.0
Devices [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Load T5 Model 

* 1. Note `use_auto_regressive=True`, argument. This is required for any models to enable text-generation.

In [3]:
model_name = 't5-small'

tokenizer = T5TokenizerTFText.from_pretrained(model_name, dynamic_padding=True, truncate=True, max_length=256)
model = T5Model.from_pretrained(model_name, use_auto_regressive=True)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

INFO:absl:Saving t5-small tokenizer to /var/folders/vq/4fxns8l55gq8_msgygbyb51h0000gn/T/tftransformers_tokenizer_cache/t5-small
INFO:absl:Loading t5-small tokenizer to /var/folders/vq/4fxns8l55gq8_msgygbyb51h0000gn/T/tftransformers_tokenizer_cache/t5-small/spiece.model


Metal device set to: Apple M1


Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712 [00:00<?, ?B/s]


Two checkpoint references resolved to different objects (<tf_transformers.models.encoder_decoder.encoder_decoder.EncoderDecoder object at 0x17b752e20> and <keras.engine.input_layer.InputLayer object at 0x17b752340>).



Two checkpoint references resolved to different objects (<tf_transformers.models.encoder_decoder.encoder_decoder.EncoderDecoder object at 0x17b752e20> and <keras.engine.input_layer.InputLayer object at 0x17b752340>).
INFO:absl:Successful ✅✅: Model checkpoints matched and loaded from /Users/sarathrnair/.cache/huggingface/hub/tftransformers__t5-small.main.699b12fe9601feda4892ca82c07e800f3c1da440/ckpt-1
INFO:absl:Successful ✅: Loaded model from tftransformers/t5-small


### Serialize and load

* The most recommended way of using a Tensorflow model is to load it after serializing.
* The speedup, especially for text generation is up to 50x times.

In [3]:
# Save as serialized
model_dir = 'MODELS/t5'
model.save_transformers_serialized(model_dir)

# Load
loaded = tf.saved_model.load(model_dir)

Metal device set to: Apple M1


### Text-Generation

* . We can pass ```tf.keras.Model``` also to ```TextDecoder```, but ```SavedModel``` this is recommended

In [4]:
decoder = TextDecoder(model=loaded)

### Greedy Decoding

In [12]:
texts = ['translate English to German: The house is wonderful and we wish to be here :)', 
         'translate English to French: She is beautiful']

inputs = tokenizer({'text': tf.constant(texts)})

predictions = decoder.decode(inputs, 
                             mode='greedy', 
                             max_iterations=64, 
                             eos_id=tokenizer.eos_token_id)
print(tokenizer._tokenizer.detokenize(tf.squeeze(predictions['predicted_ids'], axis=1)))

tf.Tensor(
[b'Das Haus ist wunderbar und wir m\xc3\xb6chten hier sein :)'
 b'Elle est belle::::'], shape=(2,), dtype=string)


### Beam Decoding

In [15]:
predictions = decoder.decode(inputs, 
                             mode='beam',
                             num_beams=3,
                             max_iterations=64,
                             eos_id=tokenizer.eos_token_id)
print(tokenizer._tokenizer.detokenize(predictions['predicted_ids'][:, 0, :]))

tf.Tensor(
[b'Das Haus ist wunderbar und wir m\xc3\xb6chten hier sein :)'
 b'Elle est belle::::'], shape=(2,), dtype=string)


### Top K Nucleus Sampling

In [17]:
predictions = decoder.decode(inputs, 
                             mode='top_k_top_p',
                             top_k=50,
                             top_p=0.7,
                             num_return_sequences=3,
                             max_iterations=64,
                             eos_id=tokenizer.eos_token_id)
print(tokenizer._tokenizer.detokenize(predictions['predicted_ids'][:, 0, :]))

tf.Tensor(
[b'Das Haus ist wunderbar und wir m\xc3\xb6chten hier sein :)'
 b'Elle est belle::::'], shape=(2,), dtype=string)


### Advanced Serialization (include preprocessing + Decoding Together)

* What if we can bundle all this into a single model and serialize it ?

In [21]:
model_dir = 'MODELS/t5_serialized/'
# Load Auto Regressive Version
model = T5Model.from_pretrained(model_name=model_name, use_auto_regressive=True)
# Assume we are doing beam decoding
text_generation_kwargs = {'mode': 'beam', 
                         'num_beams': 3,
                          'max_iterations': 32,
                          'eos_id': tokenizer.eos_token_id
                         }
# TextDecoderSerializable - makes decoding serializable
decoder = TextDecoderSerializable(model=model, **text_generation_kwargs)
# TextGenerationChainer - joins tokenizer + TextDecoderSerializable
model_fully_serialized = TextGenerationChainer(tokenizer.get_model(), decoder)
model_fully_serialized = model_fully_serialized.get_model()
# Save as saved_model
model_fully_serialized.save_serialized(model_dir, overwrite=True)


Two checkpoint references resolved to different objects (<tf_transformers.models.encoder_decoder.encoder_decoder.EncoderDecoder object at 0x38790b400> and <keras.engine.input_layer.InputLayer object at 0x3827cc880>).



Two checkpoint references resolved to different objects (<tf_transformers.models.encoder_decoder.encoder_decoder.EncoderDecoder object at 0x38790b400> and <keras.engine.input_layer.InputLayer object at 0x3827cc880>).
INFO:absl:Successful ✅✅: Model checkpoints matched and loaded from /Users/sarathrnair/.cache/huggingface/hub/tftransformers__t5-small.main.699b12fe9601feda4892ca82c07e800f3c1da440/ckpt-1
INFO:absl:Successful ✅: Loaded model from tftransformers/t5-small


Using default `decoder_start_token_id` 0 from the model


INFO:tensorflow:Assets written to: MODELS/t5_serialized/assets


INFO:tensorflow:Assets written to: MODELS/t5_serialized/assets


### Load Advanced Model and Generate text

* How nice is it? All done by model, no overhead of anything (tokenization, decoding, generating)

* 1. TextDecoderSerializable - very advances serializable decoder written in pure tensorflow ops
* 2. TextGenerationChainer -  very simple ```tf.keras.layers.Layer``` wrapper.

In [22]:
loaded = tf.saved_model.load(model_dir)
model = loaded.signatures['serving_default']

In [23]:
texts = ['translate English to German: The house is wonderful and we wish to be here :)', 
         'translate English to French: She is beautiful']

predictions = model(**{'text': tf.constant(texts)})

In [25]:
print(predictions['decoded_text'])

tf.Tensor(
[b'Das Haus ist wunderbar und wir m\xc3\xb6chten hier sein :)'
 b'Elle est belle::::'], shape=(2,), dtype=string)
